# <font style="color:blue">Pretrained Classification Models in Torchvision</font>
In this notebook, we will learn how to work with the state-of-the-art models available in Torchvision package. 

We have already seen the architectures of some important models like AlexNet, VGGNet, ResNet, etc in the previous section. We will see how these models can be imported in our code using torchvision.

In [ ]:
# Import torch and torchvision modules
import torch
from torchvision import models

import ssl

In [ ]:
# Use this line only when you are using vocareum for this notebook. 
# You can simply comment out this line in case you are using Colab or your own local system.
ssl._create_default_https_context = ssl._create_unverified_context

## <font style="color:blue">1. List the Models </font>
We have imported the models module from torchvision. Now, let's see what are the available models.

In [ ]:
# Print the models available in torchvision
print(dir(models))

Wow! There are so many models present in torchvision. 

You can see that apart from the classification models like AlexNet, DenseNet, GoogleNet, there are other models like `detection`, `segmentation`, `video`. We will discuss only Classification models in this notebook.

You can also see that there are different versions of the same model, e.g. vgg11_bn and vgg11 which are the models with and without Batch Normalization.

## <font style="color:blue">2. Load a Model</font>
Let's load the AlexNet model. It has 5 convolutional layers and 3 fully connected layers. You should load other models and see how they look.

In [ ]:
# Load alexnet model
alexnet = models.alexnet(pretrained=True)

print(alexnet)

# Put our model in eval mode
alexnet.eval()

## <font style="color:blue">3. Load the classes mapping file </font>

The models are all trained on the ImageNet dataset. However, the output of the network does not say anything about the class name. It just produces numbers as output. We can find the class to which the object belongs which is a number between 0 to 999. So, we need a mapping file from these numbers to the actual classes. 

We download the file and read it in a variable named `classes`.

In [ ]:
# Download classes text file
!wget https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt --no-check-certificate
    
# no-check-certificate flag is only for running the notebook on vocareum.

# Load labels
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]


Let us print the first 5 classes in the file

In [ ]:
classes[:5]

## <font style="color:blue">4. Load the image </font>
Let's download an image of a panda from wikipedia and load the downloaded image using the PIL library.

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Grosser_Panda.JPG/800px-Grosser_Panda.JPG --no-check-certificate -O test.jpg

# no-check-certificate flag is only for running the notebook on vocareum.
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
img = Image.open("test.jpg")
plt.imshow(img)

## <font style="color:blue">5. Apply Transforms</font>
We will use the transforms module to apply some transformations to the input image before feeding it to the network.

### <font style="color:green">Why do we need transforms?</font>
We all know that the models are trained on the huge datasets. But you should also know that the data used while training is not used as is and it is pre-processed using some transformations. 

For example, the images used while training may be of different sizes. But the network accepts fixed sized inputs. Thus, the images need to be resized so that they can be fed to the network. 

Similarly, it is a standard practice to normalize the intensity values to lie between [0,1]. 

Thus, we use transforms to make the input test images compatible with what the network expects!

### <font style="color:green">What transforms do we need for using pretrained models? </font>
From PyTorch docs:
> All pre-trained models expect input images normalized in the same way, i.e. **mini-batches** of 3-channel RGB images of shape (3 x H x W), where **H and W are expected to be at least 224**. The images have to be loaded in to a **range of [0, 1]** and then **normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].**

We can achieve the above using the following code

In [ ]:
# Specify image transformations
from torchvision import transforms

transform = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                # Ensure images are of size 224
 transforms.ToTensor(),                     # Convert the image to float tensor of range [0,1]
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                # Normalize data with the mean
 std=[0.229, 0.224, 0.225]                  # Normalize data with the std
 )])

# Apply the transform to the input image
img_t = transform(img)

In [ ]:
# Create a mini-batch 
batch_t = torch.unsqueeze(img_t, 0)

## <font style="color:blue">6. Perform Inference </font>
Once we have the mini-batch with the input image, we can pass it through the network and get the predictions.

In [ ]:
# Carry out inference
out = alexnet(batch_t)
print(out.shape)
print(out)

## <font style="color:blue">7. Decode Predictions </font>
The prediction we get from the above step does not say anything about confidence of prediction. So, we apply the softmax function to the output and convert the numbers into probability measures. 

We also sort the predictions in descending order and take the first 5 values. These are the top-5 predictions from the network for the given input image. Let's see what the network predicts for our input image. The first value is the prediction and the second value is the confidence score ( in % )

In [ ]:
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
for idx in indices[0][:5]:
    print("Class:{}, Class Name: {}, Confidence: {:.4f}%".format(idx,classes[idx], percentage[idx].item()))

You can see that the network correctly identifies the giant panda with very high confidence. 

## <font style="color:blue">8. Your Turn </font>
Now, it's your turn to play! Change the model and see how different models produce different outputs.

Also check how much time it takes for each prediction for different models.

## <font style="color:blue">9. Comparison of Models </font>
So far we have discussed how we can use pre-trained models to perform image classification but one question that we have yet to answer is how do we decide which model to choose for a particular task. In this section we will compare the pre-trained models on the basis of the following criteria:

1. **Top-1 Error**: A top-1 error occurs if the class predicted by a model with highest confidence is **not** the same as the true class.
1. **Top-5 Error**: A top-5 error occurs when the true class is not among the top 5 classes predicted by a model (sorted in terms of confidence).
1. **Inference Time on CPU**: Inference time is the time taken for model inference step on CPU.
1. **Inference Time on GPU**: Inference time is the time taken for model inference step on GPU.
1. **Model size**: Here size stands for the physical space occupied by the **.pth** file of the pre-trained model supplied by PyTorch


A **good** model shall have **low Top-1 error**, **low Top-5 error**, **low inference time** on CPU and GPU and **low model size**. Let us look at the graphs to pick the best model!

#### <font style = "color:rgb(200,0,0)">We are not sharing the code for this and you should treat it as a Practice exercise. Just take any image and use different models to perform inference on the same image with different models and record the various metrics for performance and compare.</font>

### <font style="color:green">9.1. Accuracy Comparison of Models</font>
The first criterion we are going to discuss consists of Top-1 and Top-5 errors. Top-1 error refers to errors when the top predicted class is different from the ground truth. Since the problem is rather a difficult one, there is another error measure called Top-5 error. A prediction is classified as an error if none of the top-5 predicted classes are correct.

![](https://www.learnopencv.com/wp-content/uploads/2019/06/Accuracy-Comparison-of-Models.png)

Notice from the graph that both the errors follow a similar trend. **AlexNet**  was the first attempt based on Deep Learning and there has been improvement in the error since then. **Notable** mentions are GoogLeNet, ResNet, VGGNet, ResNext.

### <font style="color:green">9.2. Inference Time Comparison</font>

Next, we will compare the models based on the time taken for model inference. One image was supplied to each model multiple times and the inference time for all the iterations was averaged. Similar process was performed for CPU and then for GPU on Google Colab. Even though there are some variations in the order, we can see that **SqueezeNet**, **ShuffleNet** and **ResNet-18** had a really low inference time, which is exactly what we want.

![](https://www.learnopencv.com/wp-content/uploads/2019/06/Model-Inference-Time-Comparison-on-CPU-ms-Lower-is-better-.png)

![](https://www.learnopencv.com/wp-content/uploads/2019/06/Model-Inference-Time-Comparison-on-GPU-ms-Lower-is-better-.png)

### <font style="color:green">9.3. Model Size Comparison</font>

A lot of times when we are using a Deep Learning model on an android or iOS device, the model size becomes a deciding factor, sometimes even more important than accuracy. **SqueezeNet** has the minimum model size (5 MB), followed by **ShuffleNet V2** (6 MB) and **MobileNet V2** (14 MB). It's obvious why these models are preferred in mobile apps utilizing deep learning.

![](https://www.learnopencv.com/wp-content/uploads/2019/06/Model-Size-Comparison.png)

### <font style="color:green">9.4. Overall Comparison</font>

We discussed about which model performed better on the basis of a particular criterion. We can squeeze all those important details in one bubble chart which we can then refer to for deciding which model to go for based on our requirements.

The x-coordinate we are using is Top-1 error (**lower is better**). The y-coordinate is the inference time on GPU in milliseconds (**lower is better**). The bubble size represents the model size (**lower is better**).

#### <font style = "color:rgb(8,133,37)">NOTE :</font>

- Smaller Bubbles are better in terms of model size.
- Bubbles near the origin are better in terms of both Accuracy and Speed.

![](https://www.learnopencv.com/wp-content/uploads/2019/06/Pre-Trained-Model-Comparison-1024x633.png)

## <font style="color:blue">10. Final Verdict</font>


1. It is clear from the above graph that **ResNet50** is the best model in terms of all three parameters ( small in size and closer to origin )
1. DenseNets and ResNext101 are expensive on inference time.
1. AlexNet and SqueezeNet have pretty high error rate.